In [1]:
import argparse
import os
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
import time

import shared.misc_utils as utils

from configs import ParseParams

#from shared import embeddings

from evaluation.benchmark import benchmark
from model.attention_agent import RLAgent

import pickle,time,os

from configs import ParseParams
from shared.graph_embedding.useful_files.gnn_film_model import GNN_FiLM_Model
from shared.graph_embedding.useful_files.number_vehicle_task import Nb_Vehicles_Task
from VRP.vrp_utils import DataGenerator

In [2]:
class Embedding(object):
    '''
    This class is the base class for embedding the input graph.
    '''
    def __init__(self,emb_type, embedding_dim):
        self.emb_type = emb_type
        self.embedding_dim = embedding_dim

        self.total_time = 0

    def __call__(self,input_pnt):
        # returns the embeded tensor. Should be implemented in child classes
        pass

In [3]:
class LinearEmbedding(Embedding):
    '''
    This class implements linear embedding. It is only a mapping 
    to a higher dimensional space.
    '''
    def __init__(self,embedding_dim,_scope=''):
        '''
        Input: 
            embedding_dim: embedding dimension
        '''

        super(LinearEmbedding,self).__init__('linear',embedding_dim)
        self.project_emb = tf.compat.v1.layers.Conv1D(embedding_dim,1,
            _scope=_scope+'Embedding/conv1d')

    def __call__(self,input_pnt):
        # emb_inp_pnt: [batch_size, max_time, embedding_dim]
        time_init = time.time()
        emb_inp_pnt = self.project_emb(input_pnt)
        # emb_inp_pnt = tf.Print(emb_inp_pnt,[emb_inp_pnt])
        self.total_time += time.time() - time_init
        return emb_inp_pnt

In [4]:
class GraphEmbedding(Embedding):
    """
    This class implement a graph embedding. The specificity is that it has already been optimized on
    another task. Implementation of transfer learning.
    """
    def __init__(self,args,data_test):
        assert args['embedding_dim'] == 30, args['embedding_dim']
        super(GraphEmbedding, self).__init__('graph',embedding_dim=args['embedding_dim'])

        self.n_nodes = args['n_nodes']
        self.embedding_dim =  args['embedding_dim']
        model_path = 'shared/graph_embedding/model_storage/' + args['task'] + '_model.pickle'
        result_dir = args['log_dir'] + '/embedding/'
        os.makedirs(result_dir)
        self.graph_model = self.restore(model_path, result_dir)
        self.graph_model.params['max_nodes_in_batch'] = args['test_size'] * self.n_nodes + 10 # We can process larger batches if we don't do training
        self.embedded_data = self(data_test)


    def __call__(self, input_data):
        """
        :param input_data: the input data as given by the env i.e. [batch_size x max_time x dim_task]
        :return: an embedding corresponding to the final node represenatation obtained via transfer learning.
        """
        time_init = time.time()
        embedded_data = self.graph_model.test(input_data)
        embedded_data = np.reshape(embedded_data,(-1,self.n_nodes,self.embedding_dim))
        self.total_time += time.time() - time_init

        return embedded_data

    @staticmethod
    def restore(saved_model_path: str, result_dir: str, run_id: str = None):
        print("Loading model from file %s." % saved_model_path)
        with open(saved_model_path, 'rb') as in_file:
            data_to_load = pickle.load(in_file)

        # model_cls, _ = name_to_model_class(data_to_load['model_class']({}))   # before...
        model_cls = GNN_FiLM_Model
        task_cls, additional_task_params = Nb_Vehicles_Task, {"data_kind":'transfer_learning'}

        if run_id is None:
            run_id = "_".join([task_cls.name(), model_cls.name(data_to_load['model_params']), time.strftime("%Y-%m-%d-%H-%M-%S"), str(os.getpid())])

        task = task_cls(data_to_load['task_params'])
        task.restore_from_metadata(data_to_load['task_metadata'])

        model = model_cls(data_to_load['model_params'], task, run_id, result_dir)
        model.load_weights(data_to_load['weights'])

        model.log_line("Loaded model from snapshot %s." % saved_model_path)

        return model

In [5]:
def test():
    args, prt = ParseParams()
    data_Gen = DataGenerator(args)
    # print(data_Gen.test_data)
    print(data_Gen.test_data.shape)

    graph_embedding = GraphEmbedding(args,data_Gen.test_data)
    data = data_Gen.get_train_next()
    graph_embedding(data)

In [ ]:
    
def load_task_specific_components(task,ups):
    '''
    This function load task-specific libraries
    '''
    if task == 'vrp':
        if ups:
            from UPS.vrp_ups_utils import DataGenerator,Env,reward_func
            from UPS.vrp_ups_attention import AttentionVRP_UPS_Actor, AttentionVRP_UPS_Critic

            AttentionActor = AttentionVRP_UPS_Actor
            AttentionCritic = AttentionVRP_UPS_Critic

        else:
            from VRP.vrp_utils import DataGenerator,Env,reward_func
            from VRP.vrp_attention import AttentionVRPActor,AttentionVRPCritic

            AttentionActor = AttentionVRPActor
            AttentionCritic = AttentionVRPCritic

    elif task == 'vrptw':
        if ups:
            from UPS.vrptw_ups_utils import DataGenerator,Env,reward_func
            from UPS.vrptw_ups_attention import AttentionVRPTW_UPS_Actor, AttentionVRPTW_UPS_Critic

            AttentionActor = AttentionVRPTW_UPS_Actor
            AttentionCritic = AttentionVRPTW_UPS_Critic
        else:
            from VRPTW.vrptw_utils import DataGenerator,Env,reward_func
            from VRPTW.vrptw_attention import AttentionVRPTWActor, AttentionVRPTWCritic

            AttentionActor = AttentionVRPTWActor
            AttentionCritic = AttentionVRPTWCritic

    else:
        raise Exception('Task is not implemented')

    return DataGenerator, Env, reward_func, AttentionActor, AttentionCritic


def load_task_specific_eval(task):
    """
    Load taks specific, dependign of tw or not
    """
    if task == 'vrp':
        from evaluation.eval_VRP import eval_google_or,eval_Clarke_Wright

        return [(eval_google_or.EvalGoogleOR,'or_tools'), (eval_Clarke_Wright.EvalClarkeWright,'Clarke_Wright')]

    elif task == 'vrptw':
        from evaluation.eval_VRPTW import eval_tw_google_or,eval_I1_heuristics

        return [(eval_tw_google_or.EvalTWGoogleOR,'or_tools_tw'),(eval_I1_heuristics.EvalI1Heuristics,'I1_heuristic')]

    else:
        raise Exception('Task is not implemented')


def main(args, prt):
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    
    # load task specific classes
    DataGenerator, Env, reward_func, AttentionActor, AttentionCritic = \
        load_task_specific_components(args['task_name'],args['ups'])

    dataGen = DataGenerator(args)
    dataGen.reset()
    env = Env(args)
    # create an RL agent
    agent = RLAgent(args,
                    prt,
                    env,
                    dataGen,
                    reward_func,
                    AttentionActor,
                    AttentionCritic,
                    is_train=args['is_train'])
    agent.Initialize(sess)

    # train or evaluate
    prev_actor_loss, prev_critic_loss = float('Inf'), float('Inf')
    actor_eps, critic_eps = 1e-2, 1e-2
    start_time = time.time()
    convergence_counter = 0
    al_file = open(args['log_dir']+"/actorLoss.txt", "w")
    cl_file = open(args['log_dir']+"/criticLoss.txt", "w")
    r_file = open(args['log_dir']+"/reward.txt", "w")

    if args['is_train']:
        prt.print_out('Training started ...')
        train_time_beg = time.time()
        for step in range(args['n_train']):
            summary = agent.run_train_step()
            _, _ , actor_loss_val, critic_loss_val, actor_gra_and_var_val, critic_gra_and_var_val,\
                R_val, v_val, logprobs_val,probs_val, actions_val, idxs_val= summary

            curr_actor_loss = np.mean(actor_loss_val)
            curr_critic_loss = np.mean(critic_loss_val)
            al_file.write( str(actor_loss_val) + '\n')
            cl_file.write(str(critic_loss_val) + '\n')
            r_file.write(str(np.mean(R_val)) + '\n')

            if abs(prev_actor_loss - curr_actor_loss) < actor_eps \
                and abs(prev_critic_loss - curr_critic_loss) < critic_eps:
                convergence_counter += 1
            else:
                convergence_counter = 0
            if convergence_counter == 10:
                prt.print_out('Converged at step {}'\
                      .format(step))
                train_time_end = time.time()-train_time_beg
                prt.print_out('Train Step: {} -- Time: {} -- Train reward: {} -- Value: {}'\
                      .format(step,time.strftime("%H:%M:%S", time.gmtime(\
                        train_time_end)),np.mean(R_val),np.mean(v_val)))
                prt.print_out('    actor loss: {} -- critic loss: {}'\
                      .format(curr_actor_loss,curr_critic_loss))
                break

            if step%args['save_interval'] == 0:
                agent.saver.save(sess,args['model_dir']+'/model.ckpt', global_step=step)

            if step%args['log_interval'] == 0:
                train_time_end = time.time()-train_time_beg
                prt.print_out('Train Step: {} -- Time: {} -- Embedding Time {} -- Train reward: {} -- Value: {}'\
                      .format(step,time.strftime("%H:%M:%S", time.gmtime(\
                        train_time_end)),time.strftime("%H:%M:%S", time.gmtime(\
                        agent.embedder_model.total_time)),np.mean(R_val),np.mean(v_val)))
                prt.print_out('    actor loss: {} -- critic loss: {}'\
                      .format(curr_actor_loss, curr_critic_loss))

                train_time_beg = time.time()
                agent.embedder_model.total_time = 0
            if step%args['test_interval'] == 0:
                agent.inference(args['infer_type'])
            prev_actor_loss = curr_actor_loss
            prev_critic_loss = curr_critic_loss

        # Save the model at the end of the training
        agent.saver.save(sess,args['model_dir']+'/model.ckpt', global_step=step)

    else: # inference
        prt.print_out('Evaluation started ...')
        agent.inference(args['infer_type'])

        all_evaluator = load_task_specific_eval(args['task_name'])

        # perform the evaluation
        list_eval = ['beam_search'] #['greedy','beam_search']
        for eval_tuple in all_evaluator:
            list_eval.append(eval_tuple[1])

            object_eval = eval_tuple[0](args,env,prt,args['min_trucks'])
            object_eval.perform_routing()
        #
        benchmark_object = benchmark.Benchmark(args,env,prt)
        # list_eval.remove('Clarke_Wright')
        # #list_eval.remove('I1_heuristic')
        benchmark_object.perform_benchmark(list_eval=list_eval)

    prt.print_out('Total time is {}'.format(time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))))
    al_file.close()
    cl_file.close()
    r_file.close()


if __name__ == "__main__":
    test()
    #assert False
    args, prt = ParseParams()
    args['is_train'] = True
    # args['infer_type'] = 'single'
    args['test_size'] = 1000
   #  args['log_dir'] = "/Users/jpoullet/Documents/MIT/Thesis/ML6867_project/VRP-RL/logs/vrp20-2019-12-05_09-28-11/"
    # args['load_path'] = "/Users/jpoullet/Documents/MIT/Thesis/ML6867_project/VRP-RL/logs/vrp50-NbTruck/model/"

    # args['data_dir'] = "drive/My Drive/VRP-RL/data"
    # args['log_dir'] = "drive/My Drive/VRP-RL/logs"
    # args['log_dir'] = "{}/{}-{}".format(args['log_dir'],args['task'], utils.get_time())
    # print(args['log_dir'])
    # args['model_dir'] = os.path.join(args['log_dir'],'model')
    #
    # args['load_path'] = "drive/My Drive/VRP-RL/logs/vrptw50-2019-11-25_01-28-09/model/"
    # print(args['model_dir'])
    # # file to write the stdout
    # try:
    #     os.makedirs(args['log_dir'])
    #     os.makedirs(args['model_dir'])
    # except:
    #     pass
    #
    # # create a print handler
    # out_file = open(os.path.join(args['log_dir'], 'results.txt'),'w+')
    # prt = utils.printOut(out_file,args['stdout_print'])

    # Random
    tf.compat.v1.disable_eager_execution()
    random_seed = args['random_seed']
    if random_seed is not None and random_seed > 0:
        prt.print_out("# Set random seed to %d" % random_seed)
        np.random.seed(random_seed)
        tf.random.set_seed(random_seed)
    tf.compat.v1.reset_default_graph()

    main(args, prt)

actor_net_lr: 0.0001
agent_type: attention
batch_size: 128
beam_width: 5
capacity: 20
critic_net_lr: 0.0001
data_dir: ./data
decode_len: 20
demand_max: 9
disable_tqdm: True
dropout: 0.1
embedding_dim: 30
embedding_graph: 2
entropy_coeff: 0.0
forget_bias: 1.0
gpu: 3
hidden_dim: 128
infer_type: batch
input_dim: 3
is_train: True
load_path: 
log_dir: logs/vrp10-2021-05-10_12-42-31
log_interval: 200
mask_glimpses: True
mask_pointer: True
max_grad_norm: 2.0
min_trucks: False
model_dir: logs/vrp10-2021-05-10_12-42-31\model
n_cust: 10
n_glimpses: 0
n_nodes: 11
n_process_blocks: 3
n_train: 260000
random_seed: 24601
rnn_layers: 1
save_interval: 1000
stdout_print: True
tanh_exploration: 10.0
task: vrp10
task_name: vrp
test_interval: 200
test_size: 1000
ups: False
use_tanh: False
Created train iterator.
Loading dataset for vrp-size-1000-len-11-test.txt...
(1000, 11, 3)
Loading model from file shared/graph_embedding/model_storage/vrp10_model.pickle.
Model has 116910 parameters.


E:\anaconda\envs\vrp\lib\site-packages\tensorflow\python\framework\indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Freshly initializing graph_model/gnn_layer_0/layer_normalization/gamma:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_0/layer_normalization/beta:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_0/layer_normalization_1/gamma:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_0/layer_normalization_1/beta:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_0/layer_normalization_2/gamma:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_0/layer_normalization_2/beta:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_1/layer_normalization_3/gamma:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_1/layer_normalization_3/beta:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_1/layer_normalization_4/gamma:0 since no saved value was found.
Freshly initializing graph_model/gnn_layer_1/